In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import NullFormatter
from scipy import stats


### Выделяем период

In [ ]:
start_date='1/1/2017'
end_date ='3/1/2017'

In [ ]:
x_date = pd.date_range(start_date,end_date)
x_d =list(x_date)


###  Мир без Дисперсии

In [ ]:
hand_amount_per_day = 10000
bet_per_hand = 50
ev_for_house = 0.027


In [ ]:
def no_swing(hand_amount_per_day,bet_per_hand,ev_for_house) -> float:
    return hand_amount_per_day*bet_per_hand*ev_for_house

In [ ]:
house_res = np.array([no_swing(hand_amount_per_day,bet_per_hand,ev_for_house) for _ in x_date])


In [ ]:
def show_gragh(x_date,house_res):
    f, ax = plt.subplots(1, 1)
    f.set_size_inches(12, 7)     
    f.set_facecolor('#eee')    
    ax.set_ylabel('Day result')
    house_res_ev = np.full_like(house_res,house_res.mean())
    ax.plot(x_date, house_res)
    ax.plot(x_date, house_res_ev)
    ax.grid()
    ax.tick_params(axis='x', labelrotation=45)
    plt.show()

In [ ]:
show_gragh(x_date,house_res)

In [ ]:
###  Добавляем качели 

In [ ]:
std_one_hand = 1.1

In [ ]:
# вычисляем качели в у.е.
def count_std_per_day(std_one_hand,hand_amount_per_day,bet_per_hand) -> float:
    return std_one_hand*(hand_amount_per_day**0.5)*bet_per_hand
std_per_day = count_std_per_day(std_one_hand,hand_amount_per_day,bet_per_hand)

In [ ]:
# создаем генератор нормально распределения, который будет обозначать насколько качает результат дня 
norm_gen = stats.norm(0,std_per_day)

In [ ]:
# генерируем случайный шум
house_res_std = np.array([no_swing(hand_amount_per_day,bet_per_hand,ev_for_house)+norm_gen.rvs() for _ in x_date])

In [ ]:
show_gragh(x_date,house_res_std)

###  Добавляем Влияние Дней Недели 

In [ ]:
ts = pd.Timestamp(x_date[0])

In [ ]:
ts

In [ ]:
ts.weekday()

In [ ]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [ ]:
week_day_coef=[0.5,0.6,0.7,0.6,1.2,1.3,1]

In [ ]:
plt.stem(days,week_day_coef)

In [ ]:
def house_res_swing_and_weekly(hand_amount_per_day,bet_per_hand,ev_for_house,std_one_hand,week_day_coef,date) -> float:
    ts = pd.Timestamp(date)
    day_coeff = week_day_coef[ts.weekday()]
    hand_this_day = hand_amount_per_day*day_coeff
    bet_this_day = bet_per_hand*day_coeff
    ev_day = bet_this_day*hand_this_day*ev_for_house
    return stats.norm(ev_day,std_one_hand*bet_per_hand*(hand_this_day**0.5)).rvs()
    

In [ ]:
house_res_std_weekle = np.array([house_res_swing_and_weekly(hand_amount_per_day,bet_per_hand,ev_for_house,std_one_hand,week_day_coef,date) for date in x_date])

In [ ]:
show_gragh(x_date,house_res_std_weekle)

###  Тренд

In [ ]:
x = np.arange(len(x_d))
y = 1.1*x/x.shape

In [ ]:
plt.plot(x,y)

In [ ]:
house_res_std_weekle_trend = house_res_std_weekle + house_res_std_weekle * y

In [ ]:
show_gragh(x_date,house_res_std_weekle_trend)